In [13]:
import pandas as pd
import os
import os.path
from sklearn.model_selection import GroupShuffleSplit
from src.utils.data import getDataPandas, getConfig
from pipe.stats import stats_analyze

In [20]:
data_config = getConfig('data')
group = data_config['data_group']

In [3]:
data = getDataPandas()
data = data.sample(frac=1, random_state=1)
data = data.drop_duplicates(subset=['PATNO', 'EVENT_ID'], keep='first').reset_index(drop=True)
data = data.drop(data[data['NUPDR3OF'] < 5].index).reset_index(drop=True)
data = data.drop(data[data['LEDD'] > 5000].index).reset_index(drop=True)

In [6]:
data['KEY'] = data['PATNO'].astype(str) + data['EVENT_ID']

In [14]:
splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=0)
split = splitter.split(data, groups=data['PATNO'])
train_inds, test_inds = next(split)

In [25]:
x = data[group['demo'] + group['clinic'] + ['PATNO']]
y = data[['CAT']]

In [26]:
x_clinic_train = x.iloc[train_inds].reset_index(drop=True)
x_clinic_test = x.iloc[test_inds].reset_index(drop=True)
y_train = y.iloc[train_inds].reset_index(drop=True)
y_test = y.iloc[test_inds].reset_index(drop=True)

In [27]:
stats_analyze(x_clinic_train, x_clinic_test, y_train, y_test, data_config)

SEX chi2 p: 0.5414803876006902
CAT chi2 p: 0.9510706628888684
AGE_AT_VISIT Normaltest p_train: [0.07505303], p_test: [0.48389882]
AGE_AT_VISIT t-test p: Ttest_indResult(statistic=array([1.54953415]), pvalue=array([0.12271054]))
NUPDR3OF Normaltest p_train: [0.01407325], p_test: [0.53859285]
NUPDR3OF ranksums p: RanksumsResult(statistic=array([0.53408144]), pvalue=array([0.59328519]))
LEDD Normaltest p_train: [2.38365408e-12], p_test: [0.16081336]
LEDD ranksums p: RanksumsResult(statistic=array([0.1754091]), pvalue=array([0.86075819]))
DURATION Normaltest p_train: [0.00553476], p_test: [0.13860719]
DURATION ranksums p: RanksumsResult(statistic=array([-0.45554005]), pvalue=array([0.64872076]))



In [29]:
x_clinic_train.describe()

,AGE_AT_VISIT,SEX,NUPDR3OF,LEDD,DURATION,PATNO
count,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000
mean,64.116763,0.670520,27.514451,604.188925,41.884393,26864.751445
std,9.248878,0.471389,12.413237,417.367176,21.673541,28148.970004
min,35.100000,0.000000,7.000000,40.000000,2.000000,3107.000000
25%,57.300000,0.000000,18.000000,300.000000,26.000000,3558.000000
50%,65.000000,1.000000,26.000000,504.000000,44.000000,4019.000000
75%,71.300000,1.000000,35.000000,765.000000,54.000000,50027.000000
max,86.300000,1.000000,62.000000,2379.500000,112.000000,149511.000000


In [30]:
x_clinic_test.describe()

,AGE_AT_VISIT,SEX,NUPDR3OF,LEDD,DURATION,PATNO
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,61.736957,0.608696,25.913043,560.032609,44.152174,21970.239130
std,9.293112,0.493435,11.796094,319.334055,23.766426,29019.844196
min,38.700000,0.000000,5.000000,30.000000,4.000000,3124.000000
25%,55.625000,0.000000,16.250000,300.000000,25.250000,3309.000000
50%,62.200000,1.000000,26.500000,489.400000,43.000000,3700.000000
75%,68.425000,1.000000,34.250000,717.500000,55.750000,40744.000000
max,77.100000,1.000000,53.000000,1396.500000,107.000000,101295.000000


In [36]:
data_config['train_index'] = train_inds.tolist()
data_config['test_index'] = test_inds.tolist()

In [37]:
import json
with open(os.path.join('pipe', 'data_config.json'), 'w+', encoding="utf-8") as f:
        json.dump(data_config, f, ensure_ascii=False, indent=4)